### LOAD LIBRARIES

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.neural_network import MLPClassifier

from sklearn.metrics import accuracy_score, classification_report


from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.preprocessing import MinMaxScaler, StandardScaler

### LOAD THE DATA

In [2]:
# Assuming the dataset is loaded into a DataFrame named data
heart_failure = pd.read_csv('heart_failure_clinical_records.csv')

heart_failure.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   age                       5000 non-null   float64
 1   anaemia                   5000 non-null   int64  
 2   creatinine_phosphokinase  5000 non-null   int64  
 3   diabetes                  5000 non-null   int64  
 4   ejection_fraction         5000 non-null   int64  
 5   high_blood_pressure       5000 non-null   int64  
 6   platelets                 5000 non-null   float64
 7   serum_creatinine          5000 non-null   float64
 8   serum_sodium              5000 non-null   int64  
 9   sex                       5000 non-null   int64  
 10  smoking                   5000 non-null   int64  
 11  time                      5000 non-null   int64  
 12  DEATH_EVENT               5000 non-null   int64  
dtypes: float64(3), int64(10)
memory usage: 507.9 KB


### SCALING THE DATA USING MINMAXSCALER

In [3]:
# Separate features and target
X = heart_failure.drop('DEATH_EVENT', axis=1)
y = heart_failure['DEATH_EVENT']

# numerical columns
numerical_features = ['age', 'creatinine_phosphokinase', 'ejection_fraction',
                      'platelets', 'serum_creatinine', 'serum_sodium', 'time']

# categorical columns
categorical_features = ['anaemia', 'diabetes', 'high_blood_pressure', 'sex', 'smoking']

# Separate the columns to scale and the rest
X_numerical = X[numerical_features]
X_categorical = X[categorical_features]

# Apply one-hot encoding to categorical features
#X_categorial = pd.get_dummies(X_uncoded, drop_first=True)

#Apply MinMax scaling to the continuous features
scaler = MinMaxScaler()
X_numerical = scaler.fit_transform(X_numerical)
numerical_scaled = pd.DataFrame(X_numerical, columns=numerical_features)

#Combine scaled continuous features with one-hot encoded categorical features
X_combination = pd.concat([numerical_scaled, X_categorical], axis=1)


### SPLIT THE DATASET

The X train will be used for training and validation in the `GridSearchCV` so as to determine the best parameters.

We will use the test set to validate our model with the best hyperparameter as provided by `GridSearchCV`.

In [4]:
# split the dataset
# Split the original training data into new training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_combination, y, test_size=0.3, random_state=42)

### TRAIN THE MODEL

In [5]:
clf = MLPClassifier(
    solver='sgd',
    learning_rate='constant',
    early_stopping=True,
    n_iter_no_change=20,
    verbose=0
)

# Initialize the classifier with the best learning rate, epochs, and batch size
best_learning_rate = [0.001, 0.01, 0.1]
best_epochs = range(100, 600, 100)
best_batch_size = [i*32 for i in range(1, 5)]

# Define the parameter grid for hyperparameter tuning
param_grid = {'hidden_layer_sizes': [(100,100), (100,150), (100,200)],
              'learning_rate_init': best_learning_rate,
              'max_iter': best_epochs,
              "batch_size":best_batch_size}

# Perform grid search cross-validation for remaining hyperparameters
grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='accuracy')


# x_train and y_train will be used in the cross-validation model
grid_search.fit(X_train, y_train)

/Users/soryn/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/soryn/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/soryn/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/soryn/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/soryn/anaconda3/lib/python3.1

GridSearchCV(cv=5,
             estimator=MLPClassifier(early_stopping=True, n_iter_no_change=20,
                                     solver='sgd', verbose=0),
             param_grid={'batch_size': [32, 64, 96, 128],
                         'hidden_layer_sizes': [(100, 100), (100, 150),
                                                (100, 200)],
                         'learning_rate_init': [0.001, 0.01, 0.1],
                         'max_iter': range(100, 600, 100)},
             scoring='accuracy')

In [6]:
# Get the best hyperparameters
best_params = grid_search.best_params_
print("Best Hyperparameters: ", best_params)

Best Hyperparameters:  {'batch_size': 96, 'hidden_layer_sizes': (100, 100), 'learning_rate_init': 0.1, 'max_iter': 500}


### USE THE BEST ESTIMATOR ON THE `X_test`

In [7]:
# Evaluate the best model on the test set
best_clf = grid_search.best_estimator_
y_pred_test_best = best_clf.predict(X_test)

In [8]:
# accuracy of the best estimator (model) on the test set
test_accuracy_best = accuracy_score(y_test, y_pred_test_best)
print("Best Model Test Accuracy:", test_accuracy_best)

Best Model Test Accuracy: 0.972


In [9]:
# Classification report for the best estimator (model)
print("Classification Report for Best Model:")
print(classification_report(y_test, y_pred_test_best))

Classification Report for Best Model:
              precision    recall  f1-score   support

           0       0.98      0.98      0.98      1040
           1       0.95      0.96      0.95       460

    accuracy                           0.97      1500
   macro avg       0.97      0.97      0.97      1500
weighted avg       0.97      0.97      0.97      1500



### TRAINING THE WHOLE DATASET

In [10]:
best_clf = MLPClassifier(
    hidden_layer_sizes=best_params['hidden_layer_sizes'],
    learning_rate_init=best_params['learning_rate_init'],
    max_iter=best_params['max_iter'],
    batch_size=best_params['batch_size'],
    solver='sgd',
    learning_rate='constant',
    early_stopping=True,
    n_iter_no_change=20,
    verbose=0
)

best_clf.fit(X_combination, y)

# Save the final model
import joblib
joblib.dump(best_clf, 'final_best_model.pkl')
print("Final model saved to final_best_model.pkl")

Final model saved to final_best_model.pkl


In [11]:
from sklearn.model_selection import cross_val_score

# Perform cross-validation and print the mean accuracy
cv_scores = cross_val_score(best_clf, X_combination, y, cv=5, scoring='accuracy')
print("Cross-validation accuracy scores:", cv_scores)
print("Mean cross-validation accuracy:", cv_scores.mean())


Cross-validation accuracy scores: [0.907 0.983 0.986 0.999 0.996]
Mean cross-validation accuracy: 0.9742000000000001


In [12]:
# Load the final model from the file
final_model = joblib.load('final_best_model.pkl')


new_data = np.array([[0.872727, 0.992498, 0.879697, 0.878833, 0.099888, 0.67714, 0.298932, 0, 0, 0, 1, 1],
                    [0.4754545, 0.104210, 0.1686667, 0.3789314, 0.255618, 0.485714, 0.722420, 0, 0, 0, 1, 0]])
    


# Make predictions
predictions = final_model.predict(new_data)
print("Predictions on new data:", predictions)


Predictions on new data: [1 0]


/Users/soryn/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(


In [24]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Assuming the heart_failure DataFrame is already loaded

# Separate features and target
X = heart_failure.drop('DEATH_EVENT', axis=1)
y = heart_failure['DEATH_EVENT']

# Numerical columns
numerical_features = ['age', 'creatinine_phosphokinase', 'ejection_fraction',
                      'platelets', 'serum_creatinine', 'serum_sodium', 'time']

# Categorical columns
categorical_features = ['anaemia', 'diabetes', 'high_blood_pressure', 'sex', 'smoking']

# Separate the columns to scale and the rest
X_numerical = X[numerical_features]
X_categorical = X[categorical_features]

# Apply MinMax scaling to the continuous features
scaler = MinMaxScaler()
X_numerical_scaled = scaler.fit_transform(X_numerical)
numerical_scaled = pd.DataFrame(X_numerical_scaled, columns=numerical_features)

# Combine scaled continuous features with categorical features
X_combination = pd.concat([numerical_scaled, X_categorical.reset_index(drop=True)], axis=1)

# Example input data for a 93-year-old male
input_data = {
    'age': 28,
    'creatinine_phosphokinase': 140,  # Example value
    'ejection_fraction': 80,  # Example value
    'platelets': 500000,  # Example value
    'serum_creatinine': 1.4,  # Example value
    'serum_sodium': 155,  # Example value
    'time': 300,  # Example value
    'anaemia': 1,  # Example value
    'diabetes': 1,  # Example value
    'high_blood_pressure': 1,  # Example value
    'sex': 0,  # Female
    'smoking': 1  # Example value
}

# Convert the input data to a DataFrame
input_df = pd.DataFrame([input_data])

# Separate numerical and categorical features from the input data
input_numerical = input_df[numerical_features]
input_categorical = input_df[categorical_features]

# Scale the numerical features using the same scaler
input_numerical_scaled = scaler.transform(input_numerical)
input_numerical_scaled_df = pd.DataFrame(input_numerical_scaled, columns=numerical_features)

# Combine scaled numerical features with categorical features
input_combination = pd.concat([input_numerical_scaled_df, input_categorical.reset_index(drop=True)], axis=1)

print("Prepared Data for Prediction:\n", input_combination)

# Make predictions
predictions = final_model.predict(input_combination)
print("Predictions on new data:", predictions)


Prepared Data for Prediction:
         age  creatinine_phosphokinase  ejection_fraction  platelets  \
0 -0.218182                  0.014927                1.0   0.575706   

   serum_creatinine  serum_sodium      time  anaemia  diabetes  \
0          0.101124           1.2  1.053381        1         1   

   high_blood_pressure  sex  smoking  
0                    1    0        1  
Predictions on new data: [0]


In [23]:
Test_heart_failure = pd.read_csv('test_data2.csv')
 
Test_heart_failure.info()
 
# numerical columns
test_numerical_features = ['Age', 'creatinine_phosphokinase', 'ejection_fraction',
                      'platelets', 'serum_creatinine', 'serum_sodium', 'time']
 
# categorical columns
test_categorical_features = ['anaemia', 'diabetes', 'high_blood_pressure', 'sex', 'smoking']
 
# Separate the columns to scale and the rest
X_numerical = Test_heart_failure[test_numerical_features]
X_categorical = Test_heart_failure[test_categorical_features]
 
# Apply one-hot encoding to categorical features
#X_categorial = pd.get_dummies(X_uncoded, drop_first=True)
 
#Apply MinMax scaling to the continuous features
scaler = MinMaxScaler()
X_numerical = scaler.fit_transform(X_numerical)
numerical_scaled = pd.DataFrame(X_numerical, columns=numerical_features)
 
#Combine scaled continuous features with one-hot encoded categorical features
X_combination_test = pd.concat([numerical_scaled, X_categorical], axis=1)

predictions = final_model.predict(X_combination_test)
print("Predictions on new data:", predictions)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Age                       13 non-null     int64  
 1   creatinine_phosphokinase  13 non-null     int64  
 2   ejection_fraction         13 non-null     int64  
 3   platelets                 13 non-null     int64  
 4   serum_creatinine          13 non-null     float64
 5   serum_sodium              13 non-null     int64  
 6   time                      13 non-null     int64  
 7   anaemia                   13 non-null     int64  
 8   diabetes                  13 non-null     int64  
 9   high_blood_pressure       13 non-null     int64  
 10  sex                       13 non-null     int64  
 11  smoking                   13 non-null     int64  
dtypes: float64(1), int64(11)
memory usage: 1.3 KB
Predictions on new data: [1 1 0 1 0 1 1 1 1 1 0 1 1]
